In [9]:
# creates caption/query GloVe-200 embeddings
import gensim
from gensim.models.keyedvectors import KeyedVectors
from cogworks_data.language import get_data_path
import re, string
from operator import add

path = get_data_path("glove.6B.200d.txt.w2v")
glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [14]:
import numpy as np

def embed_text(text):
    text = strip_punc(text).lower()
    text = text.split()
    print(text)
    
    embedded = np.zeros(200)
    for word in text:
        if word in glove:
            embedded += glove[word]
            
    return embedded

punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    return punc_regex.sub('', corpus)


print(embed_text("Hello, world!").shape)

['hello', 'world']
(200,)


In [1]:
# myNN model for embedding image descriptors
import mygrad as mg
import mynn
from mynn.layers.dense import dense
from mygrad.nnet.losses.margin_ranking_loss import margin_ranking_loss
from mynn.optimizers.sgd import SGD
from mygrad.nnet.initializers import glorot_normal
from mygrad import no_autodiff
import pickle
from cogworks_data.language import get_data_path
from pathlib import Path
from Boueny import Database
import Boueny

"""
(200, 512, 512) 
(200, 200, 200) (encoded)


cosine similarity between actual image and caption 
cosine similarity between confuser image and caption

loss function -> makes sure that actual image cosine similarity is always greater than confuser image
"""
def loss_function_accuracy(wtrueimg, wtruecap, wconfimg, margin): 
    
    simtrue = mg.einsum("ni,ni -> n", wtruecap, wtrueimg)

    simconfuser = mg.einsum("ni,ni -> n", wtruecap, wconfimg)

    #returns both loss and accuracy, loss first
    
    accuracy = np.mean(simtrue.data  > simconfuser.data)  
    
    return (mg.mean(mg.maximum(0, margin - (simtrue - simconfuser))), accuracy)


class Autoencoder:
    def __init__(self, D_full, D_hidden):
        self.encode = dense(D_full, D_hidden, weight_initializer=glorot_normal, bias=False)
    
    
    def __call__(self, x):        
        self.encode(x)
    
    
    @property
    def parameters(self):
        return self.encode.parameters

In [3]:
database = Database()
Boueny.save_database(database, "database.pkl")

loading GloVe
loading coco dataset
loading resnet18
Loading image url data
Loading captions
0.0 0.5035981555734426 1.1840233863435923 2.2690977555405216
generating caption embeddings


AttributeError: 'Database' object has no attribute 'get_image_embeddings'

In [ ]:
database = Boueny.load_database("database.pkl")

In [ ]:
model = Autoencoder(512, 200)
optim = SGD(model.parameters, learning_rate=1e-3)

batch_size = 32

for epoch_cnt in range(180):
    
    triplets = database.get_training_batch(batch_size)
    print(triplets[0][1].shape)
    print(triplets[0][2].shape)
    print(triplets[0][0].shape)
    
    triplets = [(caption_id, model(true_id), model(confusor_id)) for caption_id, true_id, confusor_id in triplets]
    
    for batch_cnt in range(0, len(dvs) // batch_size):
        triplet = triplets[batchcnt]
        
        loss = loss_function_accuracy(triplet[1], triplet[0], triplet[2], 0.25)[0]

        loss.backward()
        optim.step()

with no_autodiff:
    img_embeddings = model.encode(dvs).data